神经网络
===============

``一个神经网络训练过程包括以下几点：``

- 定义一个包含可训练参数的神经网络
- 迭代整个输入
- 通过神经网络处理输入
- 计算损失
- 反向传播梯度到神经网络的参数
- 更新网络参数：weight = weight - lr * gradient

定义神经网络
--------------

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5 * 5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # if the size is a square, you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


上面也就只定义了前向函数，然后反向函数就被自动通过autograd定义了
可以使用任何张量操作在前进函数上。

一个模型可以训练的参数通过net.parameters()返回

In [14]:
params = list(net.parameters())
print(len(params))
# print(params)
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [15]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

# 上面的代码有问题 因为格式不匹配
net.zero_grad()  # 把所有的参数梯度缓存器归零

tensor([[ 0.1665, -0.1072, -0.1111, -0.0019,  0.0040,  0.0833, -0.0284, -0.0560,
         -0.0815,  0.1287]], grad_fn=<AddmmBackward>)


In [16]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info">
            <h4>Note</h4>
            
<p> ``torch.nn`` 只支持mini-batches </p>
</div>

损失函数
--------------

损失函数的输入：模型输出（也就是我们自己的运算结果） + 目标
作用：评估我们的输出距离目标还有多远

不同的损失函数包含在nn的包里面
一个简单的例子就是nn.MSELoss，这个是用来计算均方差的。


In [17]:
output = net(input)
target = torch.randn(10)  # 给了一个随机的目标
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9730, grad_fn=<MseLossBackward>)


跟随传播路径，使用grad_fn属性，可以看到如下的计算图

input 
      -> conv2d -> relu 
      -> maxpool2d 
      -> conv2d -> relu 
      -> maxpool2d 
      -> view 
      -> linear -> relu 
      -> linear -> relu 
      -> linear
      -> MSELoss
      -> loss
      
当我们调用loss.backward()，这个图都会微分
而且所有在图中的requires_grad = True的张量都将会让他们的grad来计算累计梯度

In [18]:
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

反向传播
---------------------

实现这个，只需要使用loss.backward()

但是需要首先清空现在的梯度，不然的话会产生累积。

In [19]:
net.zero_grad() 

print("conv1.bias.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0010,  0.0081, -0.0200, -0.0064, -0.0141,  0.0029])


更新神经网络参数
----------------------

最简单的规则就是使用随机梯度下降的方式
weight = weight - lr * gradient

In [20]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [22]:
# 如果想要使用不同的更新规则，使用torch.optim来实现

import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()